In [1]:
import importlib
import time

import ddpipe
importlib.reload(ddpipe)

from ddpipe import DDClient, load_env

In [ ]:
config = load_env()
dd = DDClient(config=config)

now = int(time.time())
twelve_hours_ago = now - (3600 * 12)

In [19]:
metrics_df = dd.query_metric(query="avg:system.cpu.user{*} by {host}", since=twelve_hours_ago, until=now)
print(metrics_df.head())
print(f"Returned {len(metrics_df)} metric readings")

            timestamp     value           metric       host           scope
0 2025-10-14 17:04:00  4.884404  system.cpu.user  DZ_Laptop  host:DZ_Laptop
1 2025-10-14 17:06:00  3.022469  system.cpu.user  DZ_Laptop  host:DZ_Laptop
2 2025-10-14 17:08:00  2.887347  system.cpu.user  DZ_Laptop  host:DZ_Laptop
3 2025-10-14 17:10:00  3.281247  system.cpu.user  DZ_Laptop  host:DZ_Laptop
4 2025-10-14 17:12:00  3.276397  system.cpu.user  DZ_Laptop  host:DZ_Laptop
Returned 236 metric readings


In [20]:
logs_df = dd.query_logs(since=twelve_hours_ago, until=now)
print(logs_df.head())
print(f"Returned {len(logs_df)} logs")

                         timestamp                              message  \
0 2025-10-14 18:26:02.660000+00:00  10/14/2025 14:26:02 INFO Test log 0   
1 2025-10-14 18:26:04.676000+00:00  10/14/2025 14:26:04 INFO Test log 1   
2 2025-10-14 18:26:06.697000+00:00  10/14/2025 14:26:06 INFO Test log 2   
3 2025-10-14 18:26:08.719000+00:00  10/14/2025 14:26:08 INFO Test log 3   
4 2025-10-14 18:26:10.745000+00:00  10/14/2025 14:26:10 INFO Test log 4   

        host   service status  
0  DZ_Laptop  localapp   info  
1  DZ_Laptop  localapp   info  
2  DZ_Laptop  localapp   info  
3  DZ_Laptop  localapp   info  
4  DZ_Laptop  localapp   info  
Returned 10 logs


In [21]:
merged = dd.correlate_metrics_logs(
    metric_query="avg:system.cpu.user{*} by {host}",
    log_query="*",
    since=twelve_hours_ago,
    until=now
)
print(f"{len(merged)} logs correlated with metrics")
merged.head(10)

10 logs correlated with metrics


,timestamp,message,host,service,status,value,metric,scope
0,2025-10-14 18:26:02.660,10/14/2025 14:26:02 INFO Test log 0,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
1,2025-10-14 18:26:04.676,10/14/2025 14:26:04 INFO Test log 1,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
2,2025-10-14 18:26:06.697,10/14/2025 14:26:06 INFO Test log 2,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
3,2025-10-14 18:26:08.719,10/14/2025 14:26:08 INFO Test log 3,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
4,2025-10-14 18:26:10.745,10/14/2025 14:26:10 INFO Test log 4,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
5,2025-10-14 18:26:12.756,10/14/2025 14:26:12 INFO Test log 5,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
6,2025-10-14 18:26:14.772,10/14/2025 14:26:14 INFO Test log 6,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
7,2025-10-14 18:26:16.791,10/14/2025 14:26:16 INFO Test log 7,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
8,2025-10-14 18:26:18.807,10/14/2025 14:26:18 INFO Test log 8,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
9,2025-10-14 18:26:20.823,10/14/2025 14:26:20 INFO Test log 9,DZ_Laptop,localapp,info,10.156183,system.cpu.user,host:DZ_Laptop
